In [23]:
import os
import shutil
from openpyxl import load_workbook
import re

#######################################################################
# Program functions

# Function to extract numerical values from the given Excel column
def extract_numerical_values(column_values):
    numerical_values = []
    for value in column_values:
        match = re.search(r'\d+', value)
        if match:
            numerical_values.append(match.group())
    return numerical_values
 
 # Function gets list of all paths within the specified output directory, used to match scan numbers to output path substring
def get_subfolder_paths(folder_path):
    subfolder_paths = [os.path.join(folder_path, item) for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item).replace(os.path.sep, '/'))]
    return subfolder_paths

# Function gets full path to output directory by matching numerical substring in the list of output filepaths to a scan number
def find_matching_folder(target_string, base_directory):
    for root, dirs, files in os.walk(base_directory):
        for dir_name in dirs:
            if target_string in dir_name:
                return os.path.abspath(os.path.join(root, dir_name)).replace(os.path.sep, '/')

#######################################################################
# Gets the user input for Excel file, column index, and directory paths

excel_file_path = 'C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/T7-Summary.xlsx'
column_index = 4  # Column index (0-based) for column E
start_row = 2     # Row index (0-based) for starting row
end_row = 81      # Row index (0-based) for ending row
src_directory = 'C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/source'
#src_directory = 'X:/Data/MESAL-3'
dest_directory = 'C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/destination'
#dest_directory = 'Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done'
#######################################################################
# Loads the Excel file and extracts numerical values from the specified column

wb = load_workbook(excel_file_path)
ws = wb.worksheets[1]  # Assuming the second sheet is at index 1
#ws = wb.active
column_values = [str(ws.cell(row=row, column=column_index + 1).value) for row in range(start_row, end_row + 1)]
numerical_values = extract_numerical_values(column_values)

#######################################################################

subfolder_list = get_subfolder_paths(dest_directory)

#######################################################################
# Recursively searches for folders in the source directory that match the numerical values and copies the contents

# Loops through provided source directory ('Data'), finds number match to excel number, outputs copy of files in
# source 'seg_net' directory into a new 'seg_net' directory in the output directory ('Done')
for root, dirs, files in os.walk(src_directory):
    for dir_name in dirs:
        for num_value in numerical_values:
            match_found = False
            if num_value in dir_name:
                match_found = True
                print('============================')
                print('Finding match for case: ' + num_value)
                print('Comparing to dir: ' + dir_name)
                src_subfolder = os.path.join(root, dir_name, 'seg_net').replace(os.path.sep, '/')
                print(src_subfolder)
                dest_subfolder = find_matching_folder(num_value, dest_directory)
                print(dest_subfolder)
                dest_dir = os.path.basename(dest_subfolder)
                print(dest_dir)

                count = 1
                
                print('Copying files from: ' + dir_name + ' -> ' + dest_dir)

                try:
                    
                    for item in os.listdir(src_subfolder):
                        source_item = os.path.join(src_subfolder, item).replace(os.path.sep, '/')
                        destination_folder = os.path.join(dest_subfolder, 'seg_net').replace(os.path.sep, '/')
                        destination_item = os.path.join(dest_subfolder, 'seg_net', item).replace(os.path.sep, '/')
                        # Ensure the destination folder exists, if not, 'seg_net' directory is created before copying
                        os.makedirs(destination_folder, exist_ok=True)

                        if os.path.isfile(source_item):
                            shutil.copy(source_item, destination_item)
                            print('[' + str(count) + '] ' + source_item + ' Copied to -> ' + destination_item)
                            count += 1                   

                except PermissionError as e:
                    print(f"Permission error: {e}")

                break

            if match_found:
                print('Match was alread found')
                continue
            #else:
            #    continue
#######################################################################

Finding match for case: 5010098
Comparing to dir: MESAL-5010098-E3
C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/source/MESAL-5010098-E3/seg_net
C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/destination/Case-1-MESAL-5010098-E3
Case-1-MESAL-5010098-E3
Copying files from: MESAL-5010098-E3 -> Case-1-MESAL-5010098-E3
[1] C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/source/MESAL-5010098-E3/seg_net/MESAL-5010098-E3_FRC_0.75mm_VNC_Int16.fissure.nii.gz Copied to -> C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/destination/Case-1-MESAL-5010098-E3/seg_net/MESAL-5010098-E3_FRC_0.75mm_VNC_Int16.fissure.nii.gz
[2] C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/source/MESAL-5010098-E3/seg_net/MESAL-5010098-E3_FRC_0.75mm_VNC_Int16.lobe.nii.gz Copied to -> C:/Users/Adam/Desktop/Programs/code_testing/excel_folder_copy/destination/Case-1-MESAL-5010098-E3/seg_net/MESAL-5010098-E3_FRC_0.75mm_VNC_Int16.lobe.nii.gz
[3] C:/Users/

TypeError: expected str, bytes or os.PathLike object, not NoneType